# Database Example  

In [1]:
# Import libraries
import aopy
from aopy.data import db
from ipywidgets import interactive, widgets
from datetime import date, datetime, timedelta
import re
from tqdm.notebook import tqdm
import numpy as np

In [2]:
# set up constants
preproc_dir = '/data/preprocessed'

## Finding a single recording

In [3]:
subject = 'beignet'
task = 'manual control'
project = 'baseline flash'
day = date(2022, 6, 11) # June 11, 2022

sessions = db.lookup_sessions(subject=subject, task_name=task, date=day, project=project)
e = sessions[-1]
print(e)
print(e.project)

2022-06-11 09:55:10.450163: beignet on manual control task, id=5624
baseline flash


In [4]:
# Filter on recordings with neural data
filter_fn = db.filter_has_neural_data('ecog')
sessions = db.lookup_sessions(subject='affi', task_desc='resting data', experimenter='leo', filter_fn=filter_fn)
print(f"{len(sessions)} sessions found with ecog data")

119 sessions found with ecog data


In [5]:
# Filter specific parameters
filter_fn = lambda x: x.get_task_param('reward_time') == 0.25
sessions = db.lookup_sessions(subject='affi', filter_fn=filter_fn)
print(f"{len(sessions)} sessions found with 250ms rewards")

78 sessions found with 250ms rewards


In [6]:
# Or use a shortcut
sessions = db.lookup_mc_sessions(subject=subject, date=('2023-06-05', '2023-06-10'))
subjects, ids, dates = db.list_entry_details(sessions)
print(subjects, ids, dates)

('beignet', 'beignet', 'beignet', 'beignet') ('9609', '9631', '9638', '9667') ('2023-06-05', '2023-06-06', '2023-06-08', '2023-06-09')


## Finding a range of dates

In [7]:
start = widgets.DatePicker(
    description='Start date',
    disabled=False
)
end = widgets.DatePicker(
    description='End date',
    disabled=False
)
display(start)
display(end)

DatePicker(value=None, description='Start date', step=1)

DatePicker(value=None, description='End date', step=1)

In [8]:
# Might want to do some additional filtering by date
subject = 'beignet'
task = 'bmi control'
project = 'linear map'
entries = db.lookup_sessions(subject=subject, task_name=task, date=(start.value, end.value), project=project)
print(f"{len(entries)} matching recordings")

53 matching recordings


## Select from a dropdown menu

In [9]:
entry_drop = widgets.Dropdown(options=entries)
display(entry_drop)

Dropdown(options=(2022-06-06 13:44:42.434702: beignet on bmi control task, id=5512, 2022-06-06 13:45:31.687762…

In [10]:
e = entry_drop.value
print(e.date)
print(e.datetime)

2022-06-06
2022-06-06 13:44:42.434702


## Select anything

In [11]:
start_date = widgets.DatePicker(value = datetime.today() - timedelta(days=30))
end_date = widgets.DatePicker(value = datetime.today())

# Filter by task and subject
entries = db.lookup_sessions(date=(start_date.value, end_date.value))
subjects = np.unique([e.subject for e in entries])
tasks = np.unique([e.task_name for e in entries])

subj_drop = widgets.Dropdown(options=subjects)
task_drop = widgets.Dropdown(options=tasks) 
entry_select = widgets.SelectMultiple(
    options=[],
    description='entries',
    disabled=False
)
entry_select.layout.width = '600px'

def update_entries(*args):
    entries = db.lookup_sessions(
        subject=subj_drop.value, 
        task_name=task_drop.value,
        date=(start_date.value,end_date.value)
    )
    entry_select.options = ["{} - {}: {}".format(e.id, e.task_desc, e.date) for i, e in enumerate(entries)]
    
start_date.observe(update_entries)
end_date.observe(update_entries)
subj_drop.observe(update_entries)
task_drop.observe(update_entries)

def select_entry(subject, task, start_date, end_date, entries):
    ids = [int(re.search(r'\d+', e).group()) for e in entries]
    return db.lookup_sessions(id=ids)


selection = interactive(select_entry,
                     subject=subj_drop,
                     task=task_drop,
                     start_date=start_date,
                     end_date=end_date,
                     entries=entry_select)

display(selection)

interactive(children=(Dropdown(description='subject', options=('affi', 'beignet', 'test'), value='affi'), Drop…

In [12]:
entries = selection.result
print(f"{len(entries)} selected")

3 selected


## Loading the data

In [13]:
for e in tqdm(entries):
    
    # Load the data
    exp_data, exp_metadata = aopy.data.load_preproc_exp_data(preproc_dir, e.subject, e.id, e.date)

    print(f"entry {e.id} has {len(exp_data['clock'])} clock cycles")

  0%|          | 0/3 [00:00<?, ?it/s]

entry 9847 has 666 clock cycles
entry 9848 has 822 clock cycles
entry 9853 has 738 clock cycles
